In [22]:
import pandas as pd
import numpy as np
import time 
import warnings
warnings.filterwarnings('ignore')

In [23]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import matplotlib.pyplot as plt

In [24]:
def rfefeature(indep_X, dep_Y, n):
    rfelist = []
    linear_model = LinearRegression()
    RF = RandomForestRegressor(criterion = 'squared_error', random_state = 0)
    SVR_model = SVR(kernel = 'linear')
    DT = DecisionTreeRegressor(random_state = 0)
    rfemodellist = [linear_model, SVR_model, DT, RF]
    for i in rfemodellist:
        print(i)
        log_rfe = RFE(i,n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_transform = log_fit.transform(indep_X)
        rfelist.append(log_rfe_transform)
    return rfelist

In [25]:
def split_scaler(indep_X, dep_Y):
    X_train, X_test, Y_train, Y_test = train_test_split(indep_X, dep_Y, test_size = 0.3, random_state = 0) 
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, Y_train, Y_test

In [26]:
def r2_score(regressor, X_test):
    Y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r_score = r2_score(Y_test, Y_pred)
    return r_score  

In [6]:
def linear(X_train, Y_train, X_test):
    regressor = LinearRegression()
    regressor.fit(X_train, Y_train)
    r_score = r2_score(regressor, X_test) 
    return r_score 

In [7]:
def svml(X_train, Y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'linear')
    regressor.fit(X_train, Y_train)
    r_score = r2_score(regressor, X_test) 
    return r_score 

In [8]:
def svmnl(X_train, Y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'rbf')
    regressor.fit(X_train, Y_train)
    r_score = r2_score(regressor, X_test) 
    return r_score   

In [9]:
def decision(X_train, Y_train, X_test):
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X_train, Y_train)
    r_score = r2_score(regressor, X_test) 
    return r_score 

In [10]:
def random(X_train, Y_train, X_test):
    regressor = RandomForestRegressor(criterion = 'squared_error', random_state = 0)
    regressor.fit(X_train, Y_train)
    r_score = r2_score(regressor, X_test) 
    return r_score   

In [11]:
def rfe_regression(acclin, accsvml, accsvmnl, accdeci, accrand):
    rfedataframe = pd.DataFrame(index = ['Linear', 'SVM', 'Decision','Random'], 
                             columns = ['Linear', 'SVML', 'SVMNL', 'Decision', 'Random'])
    for number, item in enumerate(rfedataframe.index):
        rfedataframe['Linear'][item] = acclin[number]
        rfedataframe['SVML'][item] = accsvml[number]
        rfedataframe['SVMNL'][item] = accsvmnl[number]
        rfedataframe['Decision'][item] = accdeci[number]
        rfedataframe['Random'][item] = accrand[number]
    return rfedataframe        

In [12]:
dataset = pd.read_csv('prep.csv',index_col = None)
df1 = dataset

In [13]:
df1 = pd.get_dummies(df1, dtype = int, drop_first = True)
df1

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [14]:
indep_X = df1.drop('classification_yes', axis = 1)
dep_Y = df1['classification_yes']

In [21]:
rfelist = rfefeature(indep_X, dep_Y,8)
acclin = []
accsvml = []
accsvmnl = []
accdeci = []
accrand = []

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(random_state=0)


In [27]:
for i in rfelist:
    X_train, X_test, Y_train, Y_test = split_scaler(i, dep_Y)
        
    r2_lin = linear(X_train, Y_train, X_test)
    acclin.append(r2_lin)
    
    r2_svml = svml(X_train, Y_train, X_test)
    accsvml.append(r2_svml)
    
    r2_svmnl = svmnl(X_train, Y_train, X_test)
    accsvmnl.append(r2_svmnl)
    
    r2_deci = decision(X_train, Y_train, X_test)
    accdeci.append(r2_deci)
    
    r2_rand = random(X_train, Y_train, X_test)
    accrand.append(r2_rand)

result = rfe_regression(acclin,accsvml,accsvmnl,accdeci,accrand)

In [18]:
# For K = 3
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.614226,0.485417,0.760606,0.783975,0.784258
SVM,0.614226,0.485417,0.760606,0.783975,0.784258
Decision,0.690717,0.665877,0.907013,0.828889,0.891031
Random,0.630966,0.612978,0.892595,0.857778,0.872009


In [19]:
# For K = 4
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.614226,0.485417,0.760606,0.783975,0.784258
SVM,0.614226,0.485417,0.760606,0.783975,0.784258
Decision,0.690717,0.665877,0.907013,0.828889,0.891031
Random,0.630966,0.612978,0.892595,0.857778,0.872009


In [22]:
# For K=5
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.622096,0.485375,0.765296,0.786705,0.786866
SVM,0.616518,0.486731,0.768363,0.784605,0.784542
Decision,0.693117,0.662735,0.904871,0.715556,0.890741
Random,0.677891,0.662789,0.921733,0.822222,0.877544


In [17]:
# For K=6
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.633734,0.486052,0.78754,0.819674,0.81827
SVM,0.623783,0.573232,0.795922,0.809049,0.808736
Decision,0.713615,0.690507,0.900489,0.822222,0.89898
Random,0.708425,0.687971,0.925715,0.857778,0.895293


In [20]:
# For K=7
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.638025,0.572635,0.786516,0.816771,0.815285
SVM,0.630993,0.572802,0.783481,0.815304,0.813284
Decision,0.713978,0.693422,0.932137,0.928889,0.959996
Random,0.711696,0.688855,0.92742,0.786667,0.869703


In [28]:
# For K=8
result

,Linear,SVML,SVMNL,Decision,Random
Linear,0.720711,0.706394,0.867715,0.924938,0.905564
SVM,0.710608,0.696879,0.856482,0.836009,0.900207
Decision,0.721463,0.70554,0.928382,0.822222,0.943744
Random,0.715368,0.693936,0.923086,0.822222,0.889284
